In [4]:
import os

os.environ["HF_HOME"] = "/home/vipin/dia/selfdrive/InternVL/data"
os.environ["HF_DATASETS_CACHE"] = "/home/vipin/dia/selfdrive/InternVL/data/driveLM"
from datasets import load_dataset
ds = load_dataset("OpenDriveLab/DriveLM")



DatasetNotFoundError: Dataset 'OpenDriveLab/DriveLM' is a gated dataset on the Hub. You must be authenticated to access it.

In [12]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("solesensei/solesensei_bdd100k")

print("Path to dataset files:", path)

100%|██████████| 7.61G/7.61G [02:42<00:00, 50.1MB/s]

Extracting files...


Path to dataset files: /home/vipin/.cache/kagglehub/datasets/solesensei/solesensei_bdd100k/versions/2


In [10]:
import json

input_json = "data/bdd100k/train_internvl_chat.jsonl"
output_jsonl = "data/bdd100k/train_internvl_chat1.jsonl"

def describe_scene(item):
    attrs = item.get("attributes", {})
    weather = attrs.get("weather", "unknown weather")
    scene = attrs.get("scene", "road")
    tod = attrs.get("timeofday", "daytime")

    labels = item.get("labels", [])
    objs = [lab.get("category", "object") for lab in labels]
    obj_description = ", ".join(objs) if objs else "no visible objects"

    return f"A {weather} {scene} during {tod}. The scene contains: {obj_description}."

# ---- read labels as JSONL or concatenated JSON ----
data = []
with open(input_json, "r") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        data.append(json.loads(line))

print("Loaded items:", len(data))

# ---- write InternVL chat jsonl ----
with open(output_jsonl, "w") as fo:
    for idx, item in enumerate(data):
        img_name = item["name"]  # e.g. "0a0c3694-9572f64f.jpg"
        caption = describe_scene(item)

        entry = {
            "id": idx,
            "image": img_name,   # filename only
            "conversations": [
                {"from": "human", "value": "<image>\nDescribe this driving scene."},
                {"from": "gpt",   "value": caption},
            ],
        }
        fo.write(json.dumps(entry) + "\n")

print("Saved:", output_jsonl)


Loaded items: 69863


KeyError: 'name'

In [7]:
import json

inp = "data/DriveLM/drivelm_val.jsonl"      # your current file
outp = "data/DriveLM/drivelm_val_fixed.jsonl"

with open(inp) as fi, open(outp, "w") as fo:
    for line in fi:
        j = json.loads(line)
        img = j["image"]
        img = img.replace("../nuscenes/", "nuscenes/")  # drop the ".."
        j["image"] = img
        fo.write(json.dumps(j) + "\n")

print("wrote", outp)


FileNotFoundError: [Errno 2] No such file or directory: 'data/DriveLM/drivelm_val.jsonl'

In [11]:
import json
import os
from pathlib import Path

print("=== Comprehensive DriveLM Data Check ===\n")

# 1. Check what directories exist
print("1. Directory structure in data/DriveLM/:")
drivelm_dir = Path("data/DriveLM")
if drivelm_dir.exists():
    for item in sorted(drivelm_dir.iterdir())[:20]:
        size = f"({len(list(item.iterdir()))} items)" if item.is_dir() else f"({item.stat().st_size} bytes)"
        print(f"  {item.name} {size}")
else:
    print(f"  Directory does not exist: {drivelm_dir.absolute()}")

print("\n2. Check first record in drivelm_train.jsonl:")
drivelm_jsonl = "data/DriveLM/drivelm_train.jsonl"
with open(drivelm_jsonl, 'r') as f:
    first_record = json.loads(f.readline())
    print(f"  Full first record:")
    print(json.dumps(first_record, indent=2)[:500])
    
    img_field = first_record.get("image", "")
    print(f"\n  Image field value: '{img_field}'")
    print(f"  Image field type: {type(img_field)}")

print("\n3. Try different path interpretations:")
possible_paths = [
    img_field,                                           # as-is
    f"data/DriveLM/{img_field}",                         # relative from repo root
    f"data/{img_field}",                                 # data/ + field value
    img_field.lstrip("../"),                             # strip leading ../
    img_field.replace("../nuscenes/", "nuscenes/"),    # like your fix script
]

for test_path in possible_paths:
    exists = os.path.exists(test_path)
    print(f"  {test_path:<80} {'✓ EXISTS' if exists else 'NOT FOUND'}")

print("\n4. Search for nuscenes images:")
nuscenes_dirs = list(Path("data/DriveLM").rglob("**/nuscenes"))
if nuscenes_dirs:
    for ndir in nuscenes_dirs[:5]:
        img_count = len(list(ndir.glob("**/*.jpg")))
        print(f"  Found: {ndir} ({img_count} images)")
else:
    # Try a broader search
    all_subdirs = list(Path("data/DriveLM").rglob("*"))
    print(f"  Total items in DriveLM: {len(all_subdirs)}")
    jpg_files = list(Path("data/DriveLM").rglob("*.jpg"))
    print(f"  JPG files found: {len(jpg_files)}")
    if jpg_files:
        print(f"  Sample image paths:")
        for jpg in jpg_files[:3]:
            print(f"    {jpg}")


=== Comprehensive DriveLM Data Check ===

1. Directory structure in data/DriveLM/:
  drivelm_nus_imgs_train (1 items)
  drivelm_nus_imgs_val (1 items)
  drivelm_train.jsonl (114829348 bytes)
  drivelm_val.jsonl (12709390 bytes)
  make_drivelm_internvl.py (2356 bytes)
  v1_1_train_nus.json (192961829 bytes)
  v1_1_val_nus_q_only.json (9860626 bytes)

2. Check first record in drivelm_train.jsonl:
  Full first record:
{
  "image": "nuscenes/samples/CAM_FRONT/n008-2018-07-27-12-07-38-0400__CAM_FRONT__1532707819362404.jpg",
  "conversations": [
    {
      "from": "user",
      "value": "<img>\nExcept for the ego vehicle, what object would consider <c5,CAM_BACK,1005.0,512.7> to be most relevant to its decision?\n</img>"
    },
    {
      "from": "assistant",
      "value": "None."
    }
  ]
}

  Image field value: 'nuscenes/samples/CAM_FRONT/n008-2018-07-27-12-07-38-0400__CAM_FRONT__1532707819362404.jpg'
  Image field type: <class 'str'>

3. Try different path interpretations:
  nuscenes/s

In [13]:
import os, json
from PIL import Image

root = "data/DriveLM/drivelm_nus_imgs_train"
inp  = "data/DriveLM/drivelm_train.jsonl"
outp = "data/DriveLM/drivelm_chat_clean.jsonl"

good, bad = 0, 0
with open(inp) as fi, open(outp, "w") as fo:
    for line in fi:
        j = json.loads(line)
        path = os.path.join(root, j["image"])
        try:
            img = Image.open(path)
            img.verify()
            fo.write(json.dumps(j) + "\n")
            good += 1
        except Exception:
            bad += 1
            if bad < 20:
                print("BAD:", path)

print("kept", good, "skipped", bad)


kept 340159 skipped 0


In [ ]:
import json

path = "internvl_chat/shell/data/drivelm_finetune.json"
count = 0

with open(path, "r") as f:
    data = json.load(f)

def replace_img(obj):
    if isinstance(obj, dict):
        return {k: replace_img(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [replace_img(x) for x in obj]
    elif isinstance(obj, str):
        return obj.replace("<img>", "<image>")
        count += 1
    else:
        return obj
    print("Total replacements made:", count)

data = replace_img(data)

with open(path, "w") as f:
    json.dump(data, f)



FileNotFoundError: [Errno 2] No such file or directory: 'internvl_chat/shell/data/drivelm_finetune.jsonl'